### Imports

In [14]:
from sqlalchemy import create_engine
from sqlalchemy import inspect

from sqlalchemy import MetaData
from sqlalchemy import Table
import pandas as pd
import pprint as pp

### Create Engine

In [90]:
# create an engine to connect to the Postgres table in the aws instance
cnx = create_engine('postgresql://swami:@18.222.109.102:5432/ds2')

In [6]:
# check out the tables available in the database
inspect(cnx).get_table_names()

['inventory',
 'orders',
 'cust_hist',
 'products',
 'categories',
 'reorder',
 'customers',
 'orderlines']

In [13]:
# Create a MetaData instance
metadata = MetaData()
print(metadata.tables)

# reflect db schema to MetaData
metadata.reflect(bind=cnx)
pp.pprint(metadata.tables)

immutabledict({})
{'categories': Table('categories', MetaData(bind=None), Column('category', INTEGER(), table=<categories>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x11a57ca58>, for_update=False)), Column('categoryname', VARCHAR(length=50), table=<categories>, nullable=False), schema=None),
 'cust_hist': Table('cust_hist', MetaData(bind=None), Column('customerid', INTEGER(), ForeignKey('customers.customerid'), table=<cust_hist>, nullable=False), Column('orderid', INTEGER(), table=<cust_hist>, nullable=False), Column('prod_id', INTEGER(), table=<cust_hist>, nullable=False), schema=None),
 'customers': Table('customers', MetaData(bind=None), Column('customerid', INTEGER(), table=<customers>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x11a57c470>, for_update=False)), Column('firstname', VARCHAR(length=50), table=<customers>, nullable=False), Column('lastn

### Table schema

![Table Schema](table_schema.png "Table Schema")

### Simple INNER JOIN

Created a simple table that merges the orders and customer tables using the common key - customerid - to create a new table with the order details as well as customer details. This table would be useful to answer questions regarding customers - where they come from, what is the average income, gender distribution, age distribution etc. We can also get country level data to see what the total revenue per country etc.

In [3]:
pd.read_sql_query('''SELECT orders.orderid, 
      orders.orderdate, 
      orders.customerid, 
      orders.totalamount,
      customers.city, 
      customers.state, 
      customers.country, 
      customers.age,
      customers.income, 
      customers.gender       
FROM orders
JOIN customers
ON orders.customerid = customers.customerid
LIMIT 5''', cnx)

,orderid,orderdate,customerid,totalamount,city,state,country,age,income,gender
0,4695,2009-05-11,3,421.15,LWVIFXJ,OH,US,47,100000,M
1,11021,2009-12-15,4,139.13,HOKEXCD,MS,US,44,40000,F
2,3446,2009-04-30,8,297.42,BKSRQJE,NJ,US,67,80000,F
3,6706,2009-07-27,8,70.89,BKSRQJE,NJ,US,67,80000,F
4,2823,2009-03-06,11,428.19,ICLYPGR,PA,US,58,60000,M


### Creating a report with multiple columns

The SQL code to create a report that contains the following columns:

* orderid
* orderdate
* customerid
* totalamount
* city
* state
* country
* age
* income
* gender
* order quantity
* previous order count
* total amount spent on the previous order
* customer type (new or existing)
* order status

We can start off by creating separate tables and merging them to get the final report we need. An alternate method is to use `CREATE VIEW` instead of `CREATE TABLE`, which will map and optimize the steps to creating the necessary tables without storing them in the database. While this saves space, running the query requires all the views to run which may take some time...

For starters we can combine the orders and customers tables using a simple left join matching on the customerid columns of orders and customers. This will help create the first 10 columns required in the report.

In [ ]:
CREATE VIEW order_cust AS
SELECT orders.orderid, 
      orders.orderdate, 
      orders.customerid, 
      orders.totalamount,
      customers.city, 
      customers.state, 
      customers.country, 
      customers.age,
      customers.income, 
      customers.gender       
FROM orders
JOIN customers
ON orders.customerid = customers.customerid

In [ ]:
orderid | orderdate  | customerid | totalamount |  city   | state |  country  | age | income | gender 
---------+------------+------------+-------------+---------+-------+-----------+-----+--------+--------
       1 | 2009-11-12 |      15662 |      345.73 | IJTUYSA |       | France    |  65 |  80000 | M
       2 | 2009-01-25 |       3134 |      173.61 | QOALSZY | UT    | US        |  22 |  80000 | M
       3 | 2009-01-27 |      12253 |      128.19 | KITUGBU |       | Australia |  31 |  80000 | F
       4 | 2009-01-29 |       1396 |      411.07 | IBHOPBE | NH    | US        |  36 |  40000 | M
       5 | 2009-01-31 |      18045 |      368.45 | FZOGPEH |       | Japan     |  33 |  60000 | M
(5 rows)

We next create a table with orderid and the aggregate number of products per order and the total number of products per order.

In [ ]:
CREATE VIEW order_quant AS
SELECT orderid, 
       COUNT(prod_id) AS order_quantity     
       FROM orderlines 
       GROUP BY orderid;

In [ ]:
orderid | order_quantity 
---------+----------------
    6114 |              7
   11233 |              9
    4790 |              2
     273 |              4
   11719 |              7
(5 rows)

We next merge this table with the order_cust table using a left join matching orderid on the order_cust with the order_quant table.

In [ ]:
CREATE TABLE first_merge AS
SELECT order_cust.orderid, 
      order_cust.orderdate, 
      order_cust.customerid, 
      order_cust.totalamount,
      order_cust.city, 
      order_cust.state, 
      order_cust.country, 
      order_cust.age,
      order_cust.income, 
      order_cust.gender, 
      order_quant.order_quantity
FROM order_cust
JOIN order_quant
ON order_cust.orderid = order_quant.orderid;

In [ ]:
 orderid | orderdate  | customerid | totalamount |  city   | state | country | age | income | gender | order_quantity 
---------+------------+------------+-------------+---------+-------+---------+-----+--------+--------+----------------
    6114 | 2009-07-24 |       4281 |      257.14 | VUMMGKS | OR    | US      |  58 |  60000 | F      |              7
   11233 | 2009-12-09 |      13097 |      302.54 | FKNTZUP |       | UK      |  37 |  40000 | F      |              9
    4790 | 2009-05-06 |      14502 |      292.84 | QKDUKOK |       | Russia  |  47 |  40000 | M      |              2
     273 | 2009-01-09 |      17465 |      328.07 | GUOGRPE |       | Canada  |  45 |  80000 | M      |              4
   11719 | 2009-12-08 |       9703 |      174.20 | YUUVVXH | PA    | US      |  24 | 100000 | M      |              7
(5 rows)

In [ ]:
4. CREATE TABLE quantity_table AS
SELECT customerid, 
       COUNT(orderid) AS order_count, 
       SUM(pre_ord_price_table.amount) as total_amount 
         FROM (SELECT cust_hist.orderid, 
                                      cust_hist.customerid, 
                                      cust_hist.prod_id, 
                                      products.price,
                                      orderlines.quantity,
                                      (products.price*orderlines.quantity) AS amount
                       FROM cust_hist
                       JOIN products
                       ON cust_hist.prod_id = products.prod_id
                       JOIN orderlines
                       ON cust_hist.orderid = orderlines.orderid
                       AND cust_hist.prod_id = orderlines.prod_id) AS pre_ord_price_table
         GROUP BY customerid;
        
        
5. CREATE TABLE second_merge AS
SELECT first_merge.orderid, 
      first_merge.orderdate, 
      first_merge.customerid, 
      first_merge.totalamount,
      first_merge.city, 
      first_merge.state, 
      first_merge.country, 
      first_merge.age,
      first_merge.income, 
      first_merge.gender, 
      first_merge.order_quantity,
      quantity_table.order_count AS previous_order_count,
      quantity_table.total_amount AS total_amount_prev_ord
FROM first_merge
LEFT JOIN quantity_table
ON first_merge.customerid = quantity_table.customerid
ORDER BY 3;

6. CREATE TABLE table_one AS
SELECT *,
CASE WHEN second_merge.customerid IN (SELECT customerid FROM cust_hist) THEN 'existing'
ELSE 'new'
END as customer_type
FROM second_merge
ORDER BY orderid;

7. CREATE TABLE prod_inv AS
SELECT subtract.prod_id, 
    subtract.orderid, 
    subtract.quantity, 
    subtract.orderdate, 
    subtract.quan_in_stock,
    SUM(subtract.quantity) OVER (PARTITION BY prod_id  ROWS UNBOUNDED PRECEDING) AS sum_orders
    FROM
(SELECT order_stock.prod_id,
    order_stock.orderid,
    order_stock.quantity,
    order_stock.orderdate,
    order_stock.quan_in_stock
FROM (SELECT orderlines.prod_id,
    orderlines.orderid,
    orderlines.quantity,
    orderlines.orderdate,
    inventory.quan_in_stock
    FROM orderlines
    JOIN inventory
    ON orderlines.prod_id = inventory.prod_id
    ORDER BY 1, 4 ASC) AS order_stock) AS subtract;


8. CREATE TABLE order_status_report AS
SELECT orderid, sum,
CASE
WHEN sum = 0 THEN 'fulfilled'
WHEN sum % 100 = 0 THEN 'backordered'
ELSE 'partial'
END AS status
FROM
(SELECT orderid, SUM(order_status)
FROM
(SELECT orderid, 
       prod_id,
       CASE 
    WHEN (quan_in_stock - sum_orders) > quantity THEN 0
    WHEN (quan_in_stock - sum_orders) < quantity THEN 1
    ELSE 100
    END AS order_status
    FROM prod_inv) AS status_update
GROUP BY orderid) AS status_symbol;

9. CREATE TABLE table_final AS
SELECT table_one.orderid,
       table_one.orderdate,
       table_one.customerid,
       table_one.totalamount,
       table_one.city,
       table_one.state,
       table_one.country,
       table_one.age,
       table_one.income,
       table_one.gender,
       table_one.order_quantity,
       table_one.previous_order_count,
       table_one.total_amount_prev_ord,
       table_one.customer_type,
       order_status_report.status
FROM table_one
JOIN order_status_report
ON table_one.orderid = order_status_report.orderid
ORDER BY orderid;

10. CREATE TABLE prod_categ AS
SELECT orderlines.orderid,
    orderlines.prod_id,
    orderlines.quantity,
    products.category,
    products.price,
    quantity*price AS amount
FROM orderlines
JOIN products
ON orderlines.prod_id = products.prod_id


SELECT category, SUM(amount)
FROM prod_categ
GROUP BY 1
ORDER BY 1;


In [ ]:
 orderid | orderdate  | customerid | totalamount |  city   | state | country | age | income | gender | order_quantity | previous_order_count | total_amount_prev_ord | customer_type |  status   
---------+------------+------------+-------------+---------+-------+---------+-----+--------+--------+----------------+----------------------+-----------------------+---------------+-----------
       1 | 2009-11-12 |    1566199 |      345.73 | JIJUOKC |       | China   |  58 |  60000 | M      |              9 |                    9 |                324.85 | existing      | partial
       2 | 2009-01-25 |     313359 |      173.61 | XIQAQYK | UT    | US      |  61 |  20000 | F      |              2 |                    2 |                 94.95 | existing      | fulfilled
       3 | 2009-01-27 |    1225280 |      128.19 | AYLEIQI |       | UK      |  87 |  60000 | M      |              6 |                    6 |                189.89 | existing      | fulfilled
       4 | 2009-01-29 |     139511 |      411.07 | UAZPITF | OK    | US      |  30 |  80000 | F      |              5 |                   10 |                354.82 | existing      | fulfilled
       5 | 2009-01-31 |    1804417 |      368.45 | ZIMPBDD |       | China   |  34 | 100000 | M      |              3 |                    5 |                110.92 | existing      | fulfilled
(5 rows)

### Part 2. 

#### a: 

What percent of the sales come from new vs. existing customers?


In [23]:
pd.read_sql_query('''SELECT customer_type, 
       (SUM(totalamount) * 100 / (SELECT SUM(totalamount) FROM table_final)) as Percentage
FROM table_final
GROUP BY customer_type''', cnx)                 

,customer_type,percentage
0,existing,100.0


In [ ]:
NO NEW CUSTOMERS!

In [91]:
pd. read_sql_query('''SELECT status, 
       ROUND((SUM(order_quantity) * 100 / (SELECT SUM(order_quantity) FROM table_final)),2) as Percentage
FROM table_final
GROUP BY status''', cnx)

,status,percentage
0,fulfilled,85.97
1,partial,12.63
2,backordered,1.39


### Total revenue per category

Each category is associated with a product, while each order is comprised of multiple products. Therefore, it will not be possible for us to aggregate both categories and titles into each order ID (which is how the final report is aggregated). However, we can create an alternate report file and answer this question.

In [26]:
pd.read_sql_query('''WITH prod_categ(orderid, prod_id, quantity, category, price, amount) AS
(SELECT orderlines.orderid,
    orderlines.prod_id,
    orderlines.quantity,
    products.category,
    products.price,
    quantity*price AS amount
FROM orderlines
JOIN products
ON orderlines.prod_id = products.prod_id)
SELECT prod_categ.category, categories.categoryname, SUM(amount)
FROM prod_categ
JOIN categories
ON prod_categ.category = categories.category
GROUP BY 1,2
ORDER BY 3 DESC''', cnx)

,category,categoryname,sum
0,4,Classics,163930.51
1,12,Music,162236.43
2,9,Foreign,157259.11
3,6,Documentary,153663.13
4,10,Games,153277.09
5,16,Travel,151374.66
6,2,Animation,151133.39
7,3,Children,150973.28
8,8,Family,150755.22
9,5,Comedy,148890.14


### Monthly revenue and cumulative revenue

What is the monthly revenue and what is the cumulative revenue as of each month within a year (cumulative revenue resets at the beginning of each year)?

In [67]:
pd.read_sql_query('''SELECT txn_year, txn_month, monthly_sales,
                            SUM(monthly_sales) OVER (PARTITION BY txn_year ROWS UNBOUNDED PRECEDING) AS cumulative
                            FROM (SELECT date_part('year', orderdate) AS txn_year, 
                            date_part('month', orderdate) AS txn_month,
                            SUM(totalamount) as monthly_sales
                    FROM (SELECT 
      orders.orderdate, 
      orders.customerid, 
      orders.totalamount       
FROM orders) AS orders_table
GROUP BY 1,2
ORDER BY 1,2) sales''', cnx)

,txn_year,txn_month,monthly_sales,cumulative
0,2009.0,1.0,213541.27,213541.27
1,2009.0,2.0,220566.53,434107.80
2,2009.0,3.0,226031.20,660139.00
3,2009.0,4.0,217239.04,877378.04
4,2009.0,5.0,219991.99,1097370.03
5,2009.0,6.0,221373.26,1318743.29
6,2009.0,7.0,219403.42,1538146.71
7,2009.0,8.0,225201.27,1763347.98
8,2009.0,9.0,217360.81,1980708.79
9,2009.0,10.0,218486.26,2199195.05


Order by 1, 2: for this dataset only order by 1 would work, however if there were multiple years in the data then ordering by both the year and month columns with produce a table that arranges each year in ascending order with the months of that year organized in ascending order.

#### e

What is the distribution of sales by customer age buckets?


In [68]:
pd.read_sql_query('''SELECT age_floor,
                     CONCAT(age_floor, ' to ', age_ceiling) as age_category,
                     SUM(sales) as sales
                    FROM (SELECT FLOOR(age/10.0)*10 as age_floor,
                                 FLOOR(age/10.0)*10 + 10 as age_ceiling,
                                 SUM(totalamount) as sales
                                 FROM(SELECT order_cust.orderid, 
      order_cust.orderdate, 
      order_cust.customerid, 
      order_cust.totalamount, 
      order_cust.age,
      order_cust.income
FROM order_cust) AS orders_table
GROUP BY 1,2
ORDER BY 1,2) sales
GROUP BY 1, 2
ORDER BY 3 DESC ''', cnx)

,age_floor,age_category,sales
0,60.0,60 to 70,379675.58
1,20.0,20 to 30,368574.97
2,30.0,30 to 40,368260.39
3,80.0,80 to 90,354999.98
4,50.0,50 to 60,354278.17
5,40.0,40 to 50,351026.19
6,70.0,70 to 80,342694.32
7,10.0,10 to 20,67144.94
8,90.0,90 to 100,38095.68


### From SQL Alchemy to pandas..

In [32]:
simple_join = pd.read_sql_query('''SELECT orders.orderid, 
      orders.orderdate, 
      orders.customerid, 
      orders.totalamount,
      customers.city, 
      customers.state, 
      customers.country, 
      customers.age,
      customers.income, 
      customers.gender       
FROM orders
JOIN customers
ON orders.customerid = customers.customerid''', cnx)

In [34]:
simple_join.head()

,orderid,orderdate,customerid,totalamount,city,state,country,age,income,gender
0,1,2009-11-12,15662,345.73,IJTUYSA,,France,65,80000,M
1,2,2009-01-25,3134,173.61,QOALSZY,UT,US,22,80000,M
2,3,2009-01-27,12253,128.19,KITUGBU,,Australia,31,80000,F
3,4,2009-01-29,1396,411.07,IBHOPBE,NH,US,36,40000,M
4,5,2009-01-31,18045,368.45,FZOGPEH,,Japan,33,60000,M


#### Mean and Median of the total amount

In [48]:
# mean and median of the total amount spent on site using pandas
simple_join.totalamount.median(), simple_join.totalamount.mean()

(219.54, 218.72918500000017)

In [79]:
# Total number of rows in the table
pd.read_sql_query('''SELECT COUNT(*) FROM order_cust''', cnx)

,count
0,12000


##### Using SQL for Median - Percentile_Cont

In [84]:
pd.read_sql_query('''SELECT  
  ROUND(PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY totalamount)::numeric, 2) AS median_totalamount
FROM order_cust''', cnx)

,median_totalamount
0,219.54


##### Using SQL for median - Row_Number

In [87]:
pd.read_sql_query('''SELECT ROUND(AVG(totalamount), 2) as median_totalamount  
FROM  
(SELECT totalamount, 
      ROW_NUMBER() OVER (ORDER BY totalamount) AS rows_ascending,
      ROW_NUMBER() OVER (ORDER BY totalamount DESC) AS rows_descending
   FROM order_cust
   WHERE totalamount <> 0
) AS x
WHERE rows_ascending BETWEEN rows_descending - 1 AND rows_descending + 1''', cnx) 

,median_totalamount
0,219.54


In [36]:
# code to find the gender distribution of the buyers
simple_join.gender.value_counts(normalize=True)

F    0.506833
M    0.493167
Name: gender, dtype: float64

In [70]:
#SQL query to find the gender distribution of the 
pd.read_sql_query('''SELECT gender, 
                     ROUND(((COUNT(1) * 100)/(SELECT COUNT(*) FROM customers))) as percentage
                     FROM 
                     customers
GROUP BY 1''', cnx)

,gender,percentage
0,M,49.0
1,F,50.0


In [50]:
# Pandas code to find the mean revenue per country
simple_join.groupby('country')['totalamount'].mean()

country
Australia       222.728800
Canada          225.494938
Chile           215.484290
China           224.220220
France          213.283497
Germany         213.384935
Japan           209.943688
Russia          211.419062
South Africa    212.939609
UK              225.315434
US              220.131179
Name: totalamount, dtype: float64

In [77]:
pd.read_sql_query('''SELECT country, AVG(totalamount), SUM(totalamount) as revenue, SUM(order_quantity) as total_quantity
                     FROM first_merge
                     GROUP BY 1
                     ORDER BY 1''', cnx)

,country,avg,revenue,total_quantity
0,Australia,222.728800,133637.28,2924.0
1,Canada,225.494938,126953.65,2848.0
2,Chile,215.484290,139633.82,3406.0
3,China,224.220220,132514.15,3044.0
4,France,213.283497,130529.50,3048.0
5,Germany,213.384935,131018.35,3000.0
6,Japan,209.943688,122397.17,2798.0
7,Russia,211.419062,132982.59,3178.0
8,South Africa,212.939609,119672.06,2862.0
9,UK,225.315434,129781.69,2935.0


In [118]:
product_table = pd.read_sql_query('''SELECT products.prod_id,
                                            products.category,
                                            products.price,
                                            inventory.quan_in_stock,
                                            inventory.sales        
                                   FROM products 
                                   JOIN inventory
                                   ON products.prod_id = inventory.prod_id
                                   LIMIT 5''', cnx)

In [119]:
product_table

,prod_id,category,price,quan_in_stock,sales
0,1,14,25.99,225,130
1,2,6,20.99,275,130
2,3,6,28.99,133,122
3,4,3,14.99,144,126
4,5,3,11.99,177,137


In [120]:
pd.read_sql_query('''SELECT * FROM cust_hist LIMIT 5''', cnx)

,customerid,orderid,prod_id
0,1566199,1,19756
1,1566199,1,76823
2,1566199,1,55397
3,1566199,1,62888
4,1566199,1,51341


In [121]:
order_quant = pd.read_sql_query('''SELECT orderid, COUNT(quantity) AS order_quantity        
                                   FROM orderlines 
                                   GROUP BY orderid
                                   LIMIT 5''', cnx)

In [122]:
order_quant

,orderid,order_quantity
0,1,9
1,2,2
2,3,6
3,4,5
4,5,3


NOT TO USE. IS ALREADY A PART OF THE SQL QUERY BELOW

In [123]:
pre_ord_price_table = pd.read_sql_query('''SELECT cust_hist.orderid, 
                                                  cust_hist.customerid, 
                                                  cust_hist.prod_id, 
                                                  products.price,
                                                  orderlines.quantity,
                                                  (products.price*orderlines.quantity) AS amount
                                   FROM cust_hist
                                   JOIN products
                                   ON cust_hist.prod_id = products.prod_id
                                   JOIN orderlines
                                   ON cust_hist.orderid = orderlines.orderid
                                   AND cust_hist.prod_id = orderlines.prod_id
                                   LIMIT 5''', cnx)

In [124]:
pre_ord_price_table

,orderid,customerid,prod_id,price,quantity,amount
0,1,1566199,19756,21.99,2,43.98
1,1,1566199,76823,26.99,1,26.99
2,1,1566199,55397,24.99,2,49.98
3,1,1566199,62888,16.99,2,33.98
4,1,1566199,51341,26.99,3,80.97


In [125]:
quantity_table = pd.read_sql_query('''SELECT customerid, 
                                             COUNT(orderid) AS order_count, 
                                             SUM(pre_ord_price_table.amount) as total_amount 
                     FROM (SELECT cust_hist.orderid, 
                                                  cust_hist.customerid, 
                                                  cust_hist.prod_id, 
                                                  products.price,
                                                  orderlines.quantity,
                                                  (products.price*orderlines.quantity) AS amount
                                   FROM cust_hist
                                   JOIN products
                                   ON cust_hist.prod_id = products.prod_id
                                   JOIN orderlines
                                   ON cust_hist.orderid = orderlines.orderid
                                   AND cust_hist.prod_id = orderlines.prod_id) AS pre_ord_price_table
                     GROUP BY customerid
                     LIMIT 5''', cnx)

In [126]:
quantity_table

,customerid,order_count,total_amount
0,1,6,218.88
1,2,4,108.93
2,3,12,417.78
3,5,14,477.73
4,6,2,97.96


In [136]:
pd.read_sql_query('''SELECT subtract.prod_id, 
                            subtract.orderid, 
                            subtract.quantity, 
                            subtract.orderdate, 
                            subtract.quan_in_stock,
                            SUM(subtract.quantity) OVER (PARTITION BY prod_id  ROWS UNBOUNDED PRECEDING) AS sum_orders
                            
                            FROM
                    (SELECT order_stock.prod_id,
                            order_stock.orderid,
                            order_stock.quantity,
                            order_stock.orderdate,
                            order_stock.quan_in_stock
                     FROM (SELECT orderlines.prod_id,
                            orderlines.orderid,
                            orderlines.quantity,
                            orderlines.orderdate,
                            inventory.quan_in_stock
                            FROM orderlines
                            JOIN inventory
                            ON orderlines.prod_id = inventory.prod_id
                            ORDER BY 1, 4 ASC) AS order_stock) AS subtract
                            LIMIT 100''', cnx)



,prod_id,orderid,quantity,orderdate,quan_in_stock,sum_orders
0,1,38411,3,2009-01-01,225,3
1,1,83465,2,2009-01-18,225,5
2,1,13582,3,2009-01-20,225,8
3,1,79608,2,2009-01-23,225,10
4,1,2588,2,2009-01-27,225,12
5,1,177742,3,2009-02-09,225,15
6,1,172788,1,2009-02-13,225,16
7,1,151146,1,2009-02-17,225,17
8,1,148521,1,2009-02-21,225,18
9,1,267079,2,2009-03-05,225,20
